In [1]:
#Imports

#Google Search Lib
from google_images_download import google_images_download

#Keras layers
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

#Keras extras
from keras.models import Sequential
from keras.models import load_model

#SciKit Encoders
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#Libs Extras
import matplotlib.pyplot as plt
from random import shuffle
from pathlib import Path
from PIL import Image
import numpy as np
import warnings
import json
import h5py
import os

Using TensorFlow backend.


### Baixando as imagens necessárias para treino e teste da rede neural

- Rodar apenas uma vez para baixar as imagens, filtrar manualmente as imagens sem contexto.

In [2]:
# response = google_images_download.googleimagesdownload()


# def getImages(key):
#     arguments = {"keywords": key,
#                  "chromedriver":"./chromedriver",
#                  "limit":200,
#                  "offset":100,
#                  "print_urls":True,
#                  "output_directory":"dataset/validation/"}
#     print(arguments)
#     response.download(arguments)

# pokemonList = ["Bulbasaur","Charmander","Squirtle","Caterpie","Metapod","Butterfree","Weedle","Kakuna","Beedrill","Pidgey","Rattata","Raticate","Spearow","Fearow","Ekans","Pikachu","Sandshrew","Nidoqueen","Nidoking","Clefairy","Vulpix","Ninetales","Jigglypuff","Zubat","Oddish","Gloom","Paras","Venonat","Venomoth","Dugtrio","Meowth","Persian","Psyduck","Golduck","Mankey","Growlithe","Poliwag","Abra","Machop","Bellsprout","Tentacool","Geodude","Ponyta","Slowpoke","Magnemite","Farfetch'd","Doduo","Seel","Grimer","Shellder","Cloyster","Gastly","Onix","Drowzee","Hypno","Krabby","Voltorb","Exeggcute","Exeggutor","Cubone","Hitmonlee","Hitmonchan","Lickitung","Koffing","Rhyhorn","Chansey","Tangela","Kangaskhan","Horsea","Goldeen","Seaking","Staryu","Starmie","Mr. Mime","Scyther","Jynx","Electabuzz","Magmar","Pinsir","Tauros","Magikarp","Gyarados","Lapras","Ditto","Eevee","Vaporeon","Jolteon","Flareon","Porygon","Omanyte","Kabuto","Kabutops","Aerodactyl","Snorlax","Articuno","Zapdos","Moltres","Dratini","Dragonite","Mewtwo"]

# for pokemon in pokemonList:
#     getImages(pokemon)

### Montando caminhos das bases de treino e validação

In [3]:
root = Path(".")
trainPaths = list(root.rglob("dataset1/train/**/*"))
testPaths = list(root.rglob("dataset1/validation/**/*"))


### Convertendo os PNG para JPG

- Usar as duas celulas abaixo apenas se necessário converter as imagens com transparência

In [4]:
# for path in trainPaths:
#     if "DS_Store" not in path.stem and path.is_file():
#         im = Image.open(path)
#         rgb_im = im.convert('RGB')
#         rgb_im.save(path.parent + path.stem + ".png")

In [5]:
# for path in testPaths:
#     if "DS_Store" not in path.stem and path.is_file():
#         im = Image.open(path)
#         rgb_im = im.convert('RGB')
#         rgb_im.save(path.parent + path.stem + ".png")

### Estatisticas  dos tamanhos das imagens

In [6]:
def get_size_statistics(paths, typePath):
    heights = []
    widths = []
    img_count = 0
    for path in paths:
        if "DS_Store" not in path.stem and path.is_file():
            data = np.array(Image.open(path))
            heights.append(data.shape[0])
            widths.append(data.shape[1])
            img_count += 1
    avg_height = sum(heights) / len(heights)
    avg_width = sum(widths) / len(widths)
    print("Number of " + typePath + " images: " + str(img_count))
    print("Average Height of "+ typePath + "images: " + str(avg_height)+" px")
    print("Max Height of "+ typePath + "images: " + str(max(heights))+" px")
    print("Min Height of "+ typePath + "images: " + str(min(heights))+" px")
    print("Average Width of "+ typePath + "images: " + str(avg_width)+" px")
    print("Max Width of "+ typePath + "images: " + str(max(widths))+" px")
    print("Min Width of "+ typePath + "images: " + str(min(widths))+" px")
    print("\n")

In [7]:
warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)
get_size_statistics(trainPaths, "Train")
get_size_statistics(testPaths, "Test")

Number of Train images: 61
Average Height of Trainimages: 3264.0 px
Max Height of Trainimages: 3264 px
Min Height of Trainimages: 3264 px
Average Width of Trainimages: 1836.0 px
Max Width of Trainimages: 1836 px
Min Width of Trainimages: 1836 px


Number of Test images: 22
Average Height of Testimages: 3264.0 px
Max Height of Testimages: 3264 px
Min Height of Testimages: 3264 px
Average Width of Testimages: 1836.0 px
Max Width of Testimages: 1836 px
Min Width of Testimages: 1836 px




### OneHotEncoding das classes

In [ ]:
nomes = []
for path in trainPaths:
    if path.is_dir():
        nomes.append(path.name)
    else:
        continue

- Salvando JSON com os nomes e índices dos pokemons, para usar no retorno da predição posteriormente

In [ ]:
dictNomes = dict(zip(range(len(nomes)), nomes))

with open('pokemonList.json', 'w') as pl:
    json.dump(dictNomes, pl)

- Encoding

In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(nomes)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
labels = onehot_encoder.fit_transform(integer_encoded)

### Carregando as imagens das bases de treino e validação, aplicando os seguintes processos:
- Convertendo para escala de cinza, assim a rede vai identificar as formas e então as cores serão irrelevantes.
- Transformando todas as imagens para o mesmo tamanho.
- Transformando as imagens em uma lista, onde o primeiro elemento é a imagem em formato de np.array e o segundo é a label da imagem de acordo com a classe.
- Randomizando a lista para evitar bias na rede

In [ ]:
IMG_SIZE_Y = 652 #em pixeis
IMG_SIZE_X = 367 

def load_data(paths, labels, nomes):
    data = []
    for path in paths:
        if "DS_Store" not in path.stem and path.is_file():
            img = Image.open(path)
            img = img.convert('L')
            img = img.resize((IMG_SIZE_X, IMG_SIZE_Y), Image.ANTIALIAS)
            data.append([np.array(img), labels[nomes.index(path.parent.name)]])
            
    shuffle(data)
    return data

In [ ]:
train_data = load_data(trainPaths, labels, nomes)
test_data = load_data(testPaths, labels, nomes)

### Preparando a base para treino
- Separando em x_train (matrix de imagens) e y_train (vetor de classes).

In [ ]:
trainImages = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE_X, IMG_SIZE_Y, 1)
trainLabels = np.array([i[1] for i in train_data])

### Preparando a rede e as camadas

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(IMG_SIZE_X, IMG_SIZE_Y, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation = 'softmax')) #29 é o numero de classes, TODO: deixar dinamico para a quantidade de pokemons

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
#Treinando o modelo
model.fit(trainImages, trainLabels, batch_size = 50, epochs = 50, verbose = 1, use_multiprocessing = True)

### Verificando a perda e a precisão do modelo a cada epoch.

In [ ]:
fig = plt.figure()
plt.plot(model.history.history['loss'])
plt.legend(['Loss'], loc='upper right')
plt.title('Validation Loss vs Epoch')
plt.ylabel('Validation Loss')
plt.xlabel('Epoch')

In [ ]:
fig = plt.figure()
plt.plot(model.history.history['accuracy'])
plt.legend(['Accuracy'], loc='upper left')
plt.title('Validation Accuracy vs Epoch')
plt.ylabel('Validation Accuracy')
plt.xlabel('Epoch')

### Aplicando base de validação no modelo.

In [ ]:
testImages = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZE_X, IMG_SIZE_Y, 1)
testLabels = np.array([i[1] for i in test_data])

In [ ]:
loss, acc = model.evaluate(testImages, testLabels, verbose = 0)

In [ ]:
loss

In [ ]:
acc

### Testando com outra imagem fora da base

In [ ]:
predict = Image.open("dataset1/validation/Kakuna/85.how-to-draw-Kakuna-from-Pokemon-step-0.png")
predict = predict.convert('L')
predict = predict.resize((IMG_SIZE_X, IMG_SIZE_Y), Image.ANTIALIAS)

In [ ]:
model.predict_classes(np.array(predict).reshape(-1, IMG_SIZE, IMG_SIZE, 1))

- Salvando modelo

In [ ]:
model.save("pokedex.h5")